In [4]:
''' 
Create a network to analyse and classify MNIST Dataset
Based on Stanford Lectures and Aladdin Pearson YT
Images are handdrawn numbers 1-10
'''

# Import Libraries
import torch
import torch.nn as nn  # components of neural network and loss 
import torch.optim as optim  # optimizers
import torch.nn.functional as F  
from torch.utils.data import (
    DataLoader,
)  # can load data into batches
import torchvision.datasets as datasets  # standard datasets
import torchvision.transforms as transforms  # transforms change data


In [5]:
# much faster training on gpu if possible
# Runtime -> Change runtime type -> GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# Hyperparameters
in_channel = 1 # 1 input
num_classes = 10 # 10 possible classes
learning_rate = 0.001 # learning rate of network,  determines step of optimising process
batch_size = 64 # put data in to batches
num_epochs = 3 # how many loops/iterations of training data

In [7]:
# Load in Data
# can use standard MNIST dataset
# wont be case in further networks
# need to transform to tensor so network can process it
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [8]:
# network
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10): # input 1 image -> 1 of 10 possible classes
        super(CNN, self).__init__()
        # conv layers extract features out of input to feature maps
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=8,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        # pooling layers reduce spatial dimensions
        self.pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(1, 1),
        )
        # linear layer output to classify 
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
      # call network step by step in order of processes
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x


# Initialize network
# transfer to GPU
model = CNN().to(device)


In [9]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        # after forward pass, go backwards to update
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()


In [11]:

# load in data and check model accuracy on training/test data
def test(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    # keep track of predictions
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        # print output
        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()

In [12]:
test(train_loader, model)
test(test_loader, model)

Checking accuracy on training data
Got 58832 / 60000 with accuracy 98.05
Checking accuracy on test data
Got 9788 / 10000 with accuracy 97.88
